In [1]:
import os

import numpy as np

os.chdir('/Users/edwardchiu/Desktop/projects/baha-anime-analysis')
os.getcwd()

'/Users/edwardchiu/Desktop/projects/baha-anime-analysis'

In [126]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [156]:
df_anime = pd.read_csv('data/all_anime.csv')
df_anime.head()

,name,thumbnail,total_view,total_episode,avg_view,link,score,score_count,first_launched_date,author,director,agent,animator,types
0,青之壬生浪,https://p2.bahamut.com.tw/B/ACG/c/23/000013552...,23000.0,3,7666.666667,https://ani.gamer.com.tw/animeRef.php?sn=113677,4.8,285.0,2024-10-19,安田剛士,羽原久美子,曼迪,MAHO FILM,"['青春', '勵志', '歷史']"
1,香蕉喵遊世界,https://p2.bahamut.com.tw/B/ACG/c/00/000014150...,16000.0,4,4000.000000,https://ani.gamer.com.tw/animeRef.php?sn=113633,4.6,124.0,2024-10-14,Q-LiA,矢立恭,Ani-One,TMS Entertainment（第6工作室）×Lesprit,"['溫馨', '悠閒', '泡麵番', '闔家觀賞']"
2,平凡職業造就世界最強 第三季,https://p2.bahamut.com.tw/B/ACG/c/96/000012709...,301000.0,3,100333.333333,https://ani.gamer.com.tw/animeRef.php?sn=113635,4.9,3575.0,2024-10-14,白米良,岩永彰,木棉花,asread.,"['動作', '冒險', '奇幻', '異世界', '魔法']"
3,香格里拉・開拓異境～糞作獵手挑戰神作～ 第二季,https://p2.bahamut.com.tw/B/ACG/c/12/000013861...,492000.0,4,123000.000000,https://ani.gamer.com.tw/animeRef.php?sn=113665,5.0,5647.0,2024-10-13,硬梨菜、不二涼介,窪岡俊之、池下博紀,木棉花,C2C,"['動作', '冒險', '奇幻']"
4,魔王 2099,https://p2.bahamut.com.tw/B/ACG/c/43/000013124...,220000.0,4,55000.000000,https://ani.gamer.com.tw/animeRef.php?sn=113662,4.9,2779.0,2024-10-13,紫大悟,安藤良,木棉花,J.C.STAFF,"['動作', '奇幻', '魔法', '科幻', '職場']"


In [157]:
df_anime['first_launched_date'] = pd.to_datetime(df_anime['first_launched_date'])
df_anime['first_launched_date'] = df_anime['first_launched_date'].astype(int)

In [158]:
scaler = MinMaxScaler()
df_anime[['first_launched_date', 'total_view']] = np.log(df_anime[['first_launched_date', 'total_view']])
df_anime[['scaled_launch', 'scaled_view', 'scaled_score']] = (
    scaler.fit_transform(df_anime[['first_launched_date', 'total_view', 'score']]))

In [94]:
# Function to compute Jaccard similarity
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

In [55]:
# Create an empty list to store the results
similarity_matrix = np.zeros((len(df_anime), len(df_anime)))
similarity_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [56]:
for r in range(len(df_anime)):
    for c in range(r + 1):
        if r == c:
            similarity_matrix[r, c] = float('-inf')
        else:
            similarity = jaccard_similarity(set(df_anime.iloc[r]['types']), set(df_anime.iloc[c]['types']))
            similarity_matrix[r, c] = similarity
            similarity_matrix[c, r] = similarity

In [57]:
similarity_df = pd.DataFrame(similarity_matrix, index=df_anime['name'], columns=df_anime['name'])

In [160]:
# Specify the anime for which you want to find the top 5 similar animes
target_anime = '2.5 次元的誘惑'

# Get the row corresponding to the target anime
target_index = df_anime[df_anime['name'] == target_anime].index[0]
target_similarities = pd.DataFrame(similarity_matrix[target_index],
                                   index=df_anime['name'], columns=['similarity'])

scaled_metrics = df_anime.loc[:, ['name', 'scaled_launch', 'scaled_view', 'scaled_score']]
scaled_metrics = scaled_metrics.set_index('name')

target_similarities.loc[:, ['scaled_launch', 'scaled_view', 'scaled_score']] = scaled_metrics

target_similarities['main_metic'] = target_similarities[['similarity', 'scaled_view', 'scaled_launch']].mean(axis=1)


# Exclude the similarity with itself (which is always 1.0)
# target_similarities = target_similarities.drop(target_anime)
# 
# Sort the similarities in descending order and get the top 5 most similar animes
top_5_similar_animes = target_similarities.sort_values('main_metic', ascending=False).head(10)

# Print the top 5 similar animes
top_5_similar_animes


,similarity,scaled_launch,scaled_view,scaled_score,main_metic
name,,,,,
不時輕聲地以俄語遮羞的鄰座艾莉同學,0.866667,0.996828,0.784150,0.947368,0.882548
敗北女角太多了！,0.866667,0.997152,0.778538,1.000000,0.880785
我內心的糟糕念頭 第二季,0.866667,0.991563,0.758099,1.000000,0.872109
戀上換裝娃娃,0.733333,0.969523,0.860576,1.000000,0.854477
喜歡本大爺的竟然就妳一個？,0.866667,0.943340,0.752243,0.947368,0.854083
輝夜姬想讓人告白～天才們的戀愛頭腦戰～ 第二季,0.733333,0.949501,0.878149,1.000000,0.853661
輝夜姬想讓人告白～天才們的戀愛頭腦戰～,0.733333,0.934793,0.879824,0.973684,0.849317
我們真的學不來！,0.866667,0.937537,0.738416,0.921053,0.847540
不要欺負我，長瀞同學,0.866667,0.961477,0.713006,0.973684,0.847050


In [83]:
top_5_similar_animes

,similarity,scaled_launch,scaled_view,scaled_score,main_metic
name,,,,,
咒術迴戰 第二季,1.000000,0.971245,0.360089,0.973684,0.985623
鬼滅之刃 柱訓練篇,0.739130,0.990232,0.128353,0.973684,0.864681
膽大黨,0.678571,0.999084,0.041235,1.000000,0.838828
格萊普尼爾,0.739130,0.898779,0.072652,0.894737,0.818955
AYAKA -綾島奇譚-,0.666667,0.971001,0.005165,0.868421,0.818834
不死不運,0.652174,0.976923,0.124631,0.973684,0.814548
WIND BREAKER—防風少年—,0.619048,0.987973,0.092284,0.973684,0.803510
陰陽眼見子,0.666667,0.932112,0.113745,0.973684,0.799389
我想成為影之強者！,0.640000,0.954518,0.309598,0.973684,0.797259


In [70]:
df_anime[df_anime['name'] == '格萊普尼爾']

,name,thumbnail,total_view,total_episode,avg_view,link,score,score_count,first_launched_date,author,director,agent,animator,types,scaled_launch,scaled_view,scaled_score
793,格萊普尼爾,https://p2.bahamut.com.tw/B/ACG/c/81/000010428...,2344000.0,13,180307.692308,https://ani.gamer.com.tw/animeRef.php?sn=104281,4.6,3328.0,1586044800000000000,武田すん,米田和弘,Ani-One,PINE JAM,"['動作', '超能力', '懸疑', '血腥暴力', '靈異神怪']",0.898779,0.072652,0.894737


In [41]:
df_anime['first_launched_date'] = pd.to_datetime(df_anime['first_launched_date'])

In [42]:
df_anime['first_launched_date'].astype(int)

0       1729296000000000000
1       1728864000000000000
2       1728864000000000000
3       1728777600000000000
4       1728777600000000000
               ...         
1725     575942400000000000
1726     560649600000000000
1727     546307200000000000
1728     544665600000000000
1729     314064000000000000
Name: first_launched_date, Length: 1730, dtype: int64